# Balance of plant

In [1]:
using Revise
using FUSE
using Plots; gr();
global_logger(FUSE.logger);

### Initialize FPP v1_demount case
[FPP v1 demount case documentation](https://fuse.help/cases.html#FPP)

In [2]:
dd, ini, act = FUSE.init(:FPP, version=:v1_demount, init_from=:ods, do_plot=false);

┌ Warning: dd.dataset_description was skipped in IMAS data dictionary
└ @ IMASDD ~/.julia/dev/IMASDD/src/data.jl:1067


### Run Actors that will be needed for balance of plant

In [3]:
FUSE.ActorEquilibriumTransport(dd, act)
FUSE.ActorHFSsizing(dd, act)
FUSE.ActorLFSsizing(dd, act)
FUSE.ActorCXbuild(dd, act);

### Running the simple blanket actor
[ActorBlanket documentation](https://fuse.help/actors.html#Blanket)

In [4]:
FUSE.ActorBlanket(dd,act);
dd.blanket

blanket
├─ module
│  └─ 1
│     ├─ name ➡ "all"
│     └─ time_slice
│        └─ 1
│           ├─ power_incident_neutrons ➡ 9.09321e+08
│           ├─ power_incident_radiated ➡ 0
│           ├─ power_thermal_extracted ➡ 1.09118e+09
│           ├─ power_thermal_neutrons ➡ 1.09118e+09
│           ├─ power_thermal_radiated ➡ 0
│           └─ tritium_breeding_ratio ➡ 1
├─ time ➡ [1]
└─ tritium_breeding_ratio ➡ [1]


### Running the divertors actor
[ActorDivertors documentation](https://fuse.help/actors.html#Divertors)

In [5]:
FUSE.ActorDivertors(dd,act)
dd.divertors

divertors
├─ divertor
│  └─ 1
│     ├─ name ➡ "all"
│     ├─ power_incident
│     │  ├─ data ➡ [2.95129e+08]
│     │  └─ time ➡ [1]
│     ├─ power_neutrals
│     │  ├─ data ➡ [0]
│     │  └─ time ➡ [1]
│     ├─ power_radiated
│     │  ├─ data ➡ [0]
│     │  └─ time ➡ [1]
│     ├─ power_recombination_neutrals
│     │  ├─ data ➡ [0]
│     │  └─ time ➡ [1]
│     └─ power_thermal_extracted
│        ├─ data ➡ [2.95129e+08]
│        └─ time ➡ [1]
└─ time ➡ [1]


### Running the balance of plant actor
[ActorBalanceOfPlant documentation](https://fuse.help/actors.html#BalanceOfPlant)

In [6]:
FUSE.ActorBalanceOfPlant(dd,act);

println("The net electrical power to the grid is $(round(dd.balance_of_plant.power_electric_net[end]/1e6,digits=1)) [MWe] \nWith Qplant = $(round(dd.balance_of_plant.Q_plant[end],digits=2)) \n")
display(dd.balance_of_plant)


The net electrical power to the grid is 304.5 [MWe] 
With Qplant = 2.22 



balance_of_plant
├─ Q_plant ➡ Function
├─ power_electric_net ➡ Function
├─ power_electric_plant_operation
│  ├─ system
│  │  ├─ 1
│  │  │  ├─ index ➡ 1
│  │  │  ├─ name ➡ "H&CD"
│  │  │  ├─ power ➡ 100-element Vector{Float64}
│  │  │  └─ subsystem
│  │  │     └─ 1
│  │  │        ├─ index ➡ 1
│  │  │        ├─ name ➡ "ec_launchers"
│  │  │        └─ power ➡ 100-element Vector{Float64}
│  │  ├─ 2
│  │  │  ├─ index ➡ 2
│  │  │  ├─ name ➡ "cryostat"
│  │  │  └─ power ➡ 100-element Vector{Float64}
│  │  ├─ 3
│  │  │  ├─ index ➡ 3
│  │  │  ├─ name ➡ "tritium_handling"
│  │  │  └─ power ➡ 100-element Vector{Float64}
│  │  ├─ 4
│  │  │  ├─ index ➡ 4
│  │  │  ├─ name ➡ "pumping"
│  │  │  └─ power ➡ 100-element Vector{Float64}
│  │  └─ 5
│  │     ├─ index ➡ 5
│  │     ├─ name ➡ "pf_active"
│  │     └─ power ➡ 100-element Vector{Float64}
│  └─ total_power ➡ Function
├─ thermal_cycle
│  ├─ power_electric_generated ➡ Function
│  ├─ system
│  │  ├─ 1
│  │  │  ├─ index ➡ 1
│  │  │  ├─ name ➡ "blanket